# Advance Convolutions


In [43]:
# Import all the required modules
import math
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

!pip install torchsummary
from torchsummary import summary

Cloning into 'ERA'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 190 (delta 64), reused 169 (delta 43), pack-reused 0
Receiving objects: 100% (190/190), 27.42 MiB | 29.22 MiB/s, done.
Resolving deltas: 100% (64/64), done.


## Normal Convolution

- Used as a **Feature Extractor**
- Most commonly used as Nvidia hardware and software is optimized for 3x3 kernels

In [37]:
class ClassicConv(nn.Module):
    """
    Model for demo PyTorch Implementation
    """
    def __init__(self):
        """
        Constructor
        """
        # Initialize the Module class
        super(ClassicConv, self).__init__()

        # Convolutional Block-1
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3, bias=False, padding=1)
        )

    def forward(self, x):
        """
        Forward pass for model training
        :param x: Input layer
        :return: Output of the model
        """
        x = self.conv_block1(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)


summary(ClassicConv(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 32, 32]             270
Total params: 270
Trainable params: 270
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.08
Params size (MB): 0.00
Estimated Total Size (MB): 0.09
----------------------------------------------------------------


---
## Pointwise Convolution
- Used to **Combine Features**
- It is also used to increase or decrease the number of channels in a layer

In [38]:
class PointConv(nn.Module):
    """
    Model for demo PyTorch Implementation
    """
    def __init__(self):
        """
        Constructor
        """
        # Initialize the Module class
        super(PointConv, self).__init__()

        # Convolutional Block-1: Increase number of channels
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=1, bias=False, padding=0)
        )

        # Convolutional Block-2: Decrease number of channels
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=3, kernel_size=1, bias=False, padding=0)
        )

    def forward(self, x):
        """
        Forward pass for model training
        :param x: Input layer
        :return: Output of the model
        """
        x = self.conv_block1(x)
        x = self.conv_block2(x)

        x = x.view(-1, 3)
        return F.log_softmax(x, dim=1)


summary(PointConv(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 32, 32]              30
            Conv2d-2            [-1, 3, 32, 32]              30
Total params: 60
Trainable params: 60
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.10
Params size (MB): 0.00
Estimated Total Size (MB): 0.11
----------------------------------------------------------------


---

## Depthwise Separable Convolution
- Used to **reduce number of parameters** in a layer
- As a feature extractor, it is 15-20% less efficient compared to the conventional convolution
- Instead of having a kernel with same number of channels as of input, each input channels is convolved with separate single channel kernel

In [39]:
class DepthConv(nn.Module):
    """
    Model for demo PyTorch Implementation
    """
    def __init__(self):
        """
        Constructor
        """
        # Initialize the Module class
        super(DepthConv, self).__init__()

        # Convolutional Block-1
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, bias=False, padding=1, groups=3),
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=1, bias=False)
        )


    def forward(self, x):
        """
        Forward pass for model training
        :param x: Input layer
        :return: Output of the model
        """
        x = self.conv_block1(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)


summary(DepthConv(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 3, 32, 32]              27
            Conv2d-2           [-1, 10, 32, 32]              30
Total params: 57
Trainable params: 57
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.10
Params size (MB): 0.00
Estimated Total Size (MB): 0.11
----------------------------------------------------------------


## Dilated/Atrous Convolutions
- Used to **increase receptive field of the network exponentially**
- Used for **dense predictions** - semantic/panoptic segmentations, super-resolution, denoising, generative art, keypoint detection, pose estimation, etc
- They help to identify the continuation of the feature and hence they are used after normal convolution layer

In [40]:
class DilatedConv(nn.Module):
    """
    Model for demo PyTorch Implementation
    """
    def __init__(self):
        """
        Constructor
        """
        # Initialize the Module class
        super(DilatedConv, self).__init__()

        # Convolutional Block-1
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3, bias=False, padding=1)
        )

        # Dilated Convolution Block-2
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=3, bias=False, padding=0, dilation=2)
        )


    def forward(self, x):
        """
        Forward pass for model training
        :param x: Input layer
        :return: Output of the model
        """
        x = self.conv_block1(x)
        x = self.conv_block2(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)


summary(DilatedConv(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 32, 32]             540
            Conv2d-2           [-1, 10, 28, 28]           1,800
Total params: 2,340
Trainable params: 2,340
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.22
Params size (MB): 0.01
Estimated Total Size (MB): 0.24
----------------------------------------------------------------


---

## Spatially Separable Convolution
- Used to **reduce total number of parameters**
- It was immensely used in different variants of Xception-Inception Networks as well as in MobileNets
- It is obsolete since mobile phone hardware is made powerful enough to handle normal convolutions

In [41]:
class SeparableConv(nn.Module):
    """
    Model for demo PyTorch Implementation
    """
    def __init__(self):
        """
        Constructor
        """
        # Initialize the Module class
        super(SeparableConv, self).__init__()

        # Convolutional Block-1: Increase number of channels
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(3, 1), bias=False, padding=0),
            nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(1, 3), bias=False, padding=0)
            )

    def forward(self, x):
        """
        Forward pass for model training
        :param x: Input layer
        :return: Output of the model
        """
        x = self.conv_block1(x)

        x = x.view(-1, 3)
        return F.log_softmax(x, dim=1)


summary(SeparableConv(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 3, 30, 32]              27
            Conv2d-2           [-1, 10, 30, 30]              90
Total params: 117
Trainable params: 117
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.09
Params size (MB): 0.00
Estimated Total Size (MB): 0.10
----------------------------------------------------------------


---

## Transpose Convolution
- Used to **increase channel size after convolution**
- Used in all the dense problems

In [42]:
class TransposeConv(nn.Module):
    """
    Model for demo PyTorch Implementation
    """
    def __init__(self):
        """
        Constructor
        """
        # Initialize the Module class
        super(TransposeConv, self).__init__()

        # Convolutional Block-1: Increase size of output channel
        self.conv_block1 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=3, out_channels=10, kernel_size=3, bias=False, stride=1, padding=0, output_padding=0)
            )

        # Convolutional Block-2: Increase size of output channel
        self.conv_block2 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=10, out_channels=10, kernel_size=3, bias=False, stride=2, padding=0, output_padding=1)
            )

    def forward(self, x):
        """
        Forward pass for model training
        :param x: Input layer
        :return: Output of the model
        """
        x = self.conv_block1(x)
        x = self.conv_block2(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)


summary(TransposeConv(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [-1, 10, 34, 34]             270
   ConvTranspose2d-2           [-1, 10, 70, 70]             900
Total params: 1,170
Trainable params: 1,170
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.46
Params size (MB): 0.00
Estimated Total Size (MB): 0.48
----------------------------------------------------------------
